In [1]:
import asyncio
import os
import json
from copy import copy, deepcopy
from pprint import pprint
from typing import List
from uuid import uuid4

import evaluate
import openai
import requests
import tiktoken
from bs4 import BeautifulSoup, Comment
from doctran import Doctran, ExtractProperty
from dotenv import load_dotenv, find_dotenv
from evaluate import load
from langchain.chat_models import ChatOpenAI
from langchain.docstore.document import Document
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import (
    OpenAIEmbeddings,
    HuggingFaceEmbeddings,
)
from langchain.llms import OpenAI
from langchain.text_splitter import (
    MarkdownTextSplitter,
    MarkdownHeaderTextSplitter,
    LineType,
    RecursiveCharacterTextSplitter,
)
from loguru import logger
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

from main import (
    extract_plan_and_content_wikipedia,
    compare_documents_content,
    compare_documents_sections,
)

_ = load_dotenv(find_dotenv())
openai.api_key = os.getenv('OPENAI_API_KEY')

llm_default = ChatOpenAI(model_name="gpt-3.5-turbo", streaming=True)
llm_16k = ChatOpenAI(model_name="gpt-3.5-turbo-16k", streaming=True)

def num_tokens_from_string(string: str, encoding_name: str = "gpt-3.5-turbo") -> int:
    """Returns the number of tokens in a text string."""
    try:
        encoding = tiktoken.get_encoding(encoding_name)
    except ValueError:
        encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def convert_to_markdown(article_dict):
    md_text = ""

    for heading, content in article_dict.items():
        # heading is of form: 'h3 Example'
        # Define the markdown equivalent for the heading level
        heading_level = "#" * int(heading[1])
        heading = heading[3:]
        # Append the heading and the content to the markdown text
        md_text += f"{heading_level} {heading}\n\n{content}\n\n"

    return md_text

In [2]:
a = [1, 2, 3]
b = [1, 2, 3, 4, 5, 6, 7]
for a_, b_ in zip(b, a):
    print(a_, b_)

1 1
2 2
3 3


In [3]:
url_1 = 'https://en.wikipedia.org/wiki/Simulated_annealing'
url_2 = 'https://en.wikipedia.org/wiki/Dual-phase_evolution'

doc_1 = await extract_plan_and_content_wikipedia(url_1)
doc_2 = await extract_plan_and_content_wikipedia(url_2)

2023-08-16 14:12:19.870 | INFO     | main:extract_plan_and_content:761 - 
	Extracting plan and content for: https://en.wikipedia.org/wiki/Simulated_annealing
2023-08-16 14:12:20.506 | INFO     | main:load_wikipedia_url:252 - Successfully downloaded content from Wikipedia page https://en.wikipedia.org/wiki/Simulated_annealing. Extracted 19 sections.
2023-08-16 14:12:20.511 | INFO     | main:divide_sections_if_too_large:294 - Dividing sections if too large in plan and section content.
2023-08-16 14:12:24.346 | INFO     | main:divide_sections_if_too_large:349 - Added 'h1 Simulated Annealing' split original heading 'h1 Simulated annealing'
2023-08-16 14:12:25.363 | INFO     | main:divide_sections_if_too_large:349 - Added 'h1 Simulated Annealing Algorithm' split original heading 'h1 Simulated annealing'
2023-08-16 14:12:26.388 | INFO     | main:divide_sections_if_too_large:349 - Added 'h3 Simulated Annealing' split original heading 'h3 Acceptance probabilities'
2023-08-16 14:12:27.283 | INF

2023-08-16 14:14:58.403 | INFO     | main:_gen_embed_section_content:399 - 1/22 - created section + content embeddings for h1 Simulated Annealing Algorithm
2023-08-16 14:15:02.937 | INFO     | main:_gen_embed_section_content:399 - 2/22 - created section + content embeddings for h2 Overview
2023-08-16 14:15:04.794 | INFO     | main:_gen_embed_section_content:399 - 3/22 - created section + content embeddings for h3 The basic iteration
2023-08-16 14:15:08.329 | INFO     | main:_gen_embed_section_content:399 - 4/22 - created section + content embeddings for h3 The neighbours of a state
2023-08-16 14:15:12.062 | INFO     | main:_gen_embed_section_content:399 - 5/22 - created section + content embeddings for h3 Simulated Annealing
2023-08-16 14:15:13.924 | INFO     | main:_gen_embed_section_content:399 - 6/22 - created section + content embeddings for h3 The annealing schedule
2023-08-16 14:15:16.362 | INFO     | main:_gen_embed_section_content:399 - 7/22 - created section + content embeddin

2023-08-16 14:16:03.971 | INFO     | main:_gen_embed_section_content:399 - 6/14 - created section + content embeddings for h3 System memory
2023-08-16 14:16:06.197 | INFO     | main:_gen_embed_section_content:399 - 7/14 - created section + content embeddings for h2 Examples
2023-08-16 14:16:08.503 | INFO     | main:_gen_embed_section_content:399 - 8/14 - created section + content embeddings for h3 Social networks
2023-08-16 14:16:10.408 | INFO     | main:_gen_embed_section_content:399 - 9/14 - created section + content embeddings for h3 Socio-economics
2023-08-16 14:16:12.353 | INFO     | main:_gen_embed_section_content:399 - 10/14 - created section + content embeddings for h3 Forest ecology
2023-08-16 14:16:14.236 | INFO     | main:_gen_embed_section_content:399 - 11/14 - created section + content embeddings for h3 Search algorithms
2023-08-16 14:16:15.892 | INFO     | main:_gen_embed_section_content:399 - 12/14 - created section + content embeddings for h2 Related processes
2023-08-1

In [6]:
doc_1['plan'][0].keys()

dict_keys(['section_id', 'section', 'content', 'section_embedding_1', 'section_embedding_2', 'content_embedding_1', 'content_embedding_2'])

In [7]:
compare_plan = compare_documents_plans(doc_1, doc_2, None)
compare_sections = compare_documents_sections(doc_1, doc_2, None)

TypeError: compare_documents_content() takes 2 positional arguments but 3 were given

In [ ]:
compare_sections

In [ ]:
compare_plan

In [ ]:
embed_ada = OpenAIEmbeddings(
    model="text-embedding-ada-002",
)

embed_e5 = HuggingFaceEmbeddings(
    model_name='intfloat/e5-base-v2',
    encode_kwargs = {'normalize_embeddings': True}
)

text_1 = 'hello'
text_2 = 'hi'
text_3 = 'England is a small country'

# e5-base-v2 requires all embeddings to be prepended with 'query: '
e51 = embed_e5.embed_query("query: " + text_1)
e52 = embed_e5.embed_query("query: " + text_2)
e53 = embed_e5.embed_query("query: " + text_3)
print(len(e51))
ada1 = embed_ada.embed_query(text_1)
ada2 = embed_ada.embed_query(text_2)
ada3 = embed_ada.embed_query(text_3)
print(len(ada1))

In [ ]:
a = [1, 2, 3]
b = [10, 20, 30]

print(np.mean([a, b], axis=0))

In [ ]:
a = [1, 2, 3, 4, 5]
b = [10, 20, 30, 40, 50]
# a = np.array(a)
# b = np.array(b)

d = dict(a=ada1, b=ada2)

# np.mean([ada1, ada2], axis=0)
np.mean([*d.values()], axis=0)

In [ ]:
url = 'https://en.wikipedia.org/wiki/Dual-phase_evolution'
url = 'https://en.wikipedia.org/wiki/Self-driving_car'
r = requests.get(url)
html_content = r.text

# Create a BeautifulSoup object
soup = BeautifulSoup(html_content, 'html.parser')

# Remove unwanted tags: script, style, [document], head, title
for element in soup(["script", "style", "head", "title", "[document]"]):
    element.decompose()

# Also remove HTML comments
for element in soup.find_all(string=lambda text: isinstance(text, Comment)):
    element.extract()

# Define the tags to find
tags = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']
found_tags = soup.find_all(tags)

# Extract tags and their associated content into a dictionary
doc_dict = {}
i = 0
for tag in found_tags:
    content = []
    next_tag = tag.find_next()

    # Look for next tags until the next header tag
    while next_tag and next_tag.name not in tags:
        # Reference section can contain both p and li tags
        if 'reference' in str(next_tag).lower() and next_tag.name in ['p', 'li']:
            # print(str(next_tag).lower())
            content.append(next_tag.get_text(strip=False))
        elif next_tag.name == 'p':
            content.append(next_tag.get_text(strip=False))
        next_tag = next_tag.find_next()

    key = f"{tag.name} {tag.get_text(strip=True)}"
    doc_dict[key] = " ".join(content)

for key in list(doc_dict.keys()):  # Using list() to avoid changing the dictionary size during iteration
    if key.endswith('[edit]'):
        new_key = key.rsplit('[edit]', 1)[0]
        doc_dict[new_key] = doc_dict.pop(key)
#     new_key = key.rstrip('\[edit\]')
#     if new_key != key:
#         doc_dict[new_key] = doc_dict.pop(key)
#
# processed = []
# for s in strings:
#     if s.endswith('[edit]'):
#         s = s.rsplit('[edit]', 1)[0]
#     processed.append(s)

del doc_dict['h2 Contents']

pprint(doc_dict, sort_dicts=False)
pprint(doc_dict.keys())

In [ ]:
doctran = Doctran(openai_api_key=os.getenv('OPENAI_API_KEY'), openai_model='gpt-3.5-turbo')

async def extract_topics(string: str):
    """Extract topics discussed in a string."""
    document = doctran.parse(content=string)
    properties = ExtractProperty(
        # name="keywords_max_three",
        # description="The 1, 2, or 3 most important keywords from the document.",
        name='title',
        description='The title of the document (max 7 words).',
        type="string",
        required=True
    )
    document = await document.extract(properties=[properties]).execute()
    return document.transformed_content

In [ ]:
pprint(doc_dict, sort_dicts=False)

In [ ]:
doc_dict.keys()

In [ ]:
final_dict: dict = {}
start_dict = copy(doc_dict)
for heading, content in start_dict.items():
    print(heading)

In [ ]:
final_dict: dict = {}
start_dict = copy(doc_dict)
for heading, content in start_dict.items():
    if 'reference' in heading.lower() or 'further reading' in heading.lower() or 'see also' in heading.lower():
        print(f"Found 'referece' in {heading}")
        final_dict[heading] = content
        continue
    num_tokens = num_tokens_from_string(content)
    max_allowed_tokens = 512
    if num_tokens <= max_allowed_tokens:
        final_dict[heading] = content
    else:
        # Split the document into smaller chunks, then add topics
        char_splitter = RecursiveCharacterTextSplitter(
            chunk_size=max_allowed_tokens,
            chunk_overlap=0,
            # ' ' separator means sometimes sentences will be cut in two to ensure
            # the chunk size is not exceeded
            separators=["\n\n", "\n", ' '],
            length_function=num_tokens_from_string,
        )
        splits: List[str] = char_splitter.split_text(content)
        for split in splits:
            # Headings are of the form h1, h2, h3 etc. we split it into more of the same level
            heading_level = int(heading[1])
            title = await extract_topics(split)
            new_heading = f"h{heading_level}_{title}"
            final_dict[new_heading] = split
            print(f'Added {new_heading} split original heading {heading}')

n_keys_start = len(start_dict.keys())
n_keys_final = len(final_dict.keys())
print(f"We started with {n_keys_start} sections and got {n_keys_final} final sections")
print(f"That's a {n_keys_final / n_keys_start:.2f}x increase in sections")
# pprint(final_dict, sort_dicts=False)

In [ ]:
pprint(list(final_dict.keys()))

In [ ]:
pprint(final_dict, sort_dicts=False)

In [ ]:
text_1 = final_dict['h2 Regulation']
text_2 = final_dict['h2 Commercialization']

# Normalized by default
embed_ada = OpenAIEmbeddings(
    model="text-embedding-ada-002",
)

embed_e5 = HuggingFaceEmbeddings(
    model_name='intfloat/e5-base-v2',
    encode_kwargs = {'normalize_embeddings': True}
)

# e5-base-v2 requires all embeddings to be prepended with 'query: '
e51 = embed_e5.embed_query("query: " + text_1)
e52 = embed_e5.embed_query("query: " + text_2)
print(len(e51))
ada1 = embed_ada.embed_query(text_1)
ada2 = embed_ada.embed_query(text_2)
print(len(ada1))

In [ ]:
cosine_similarity([e51], [e51])[0][0]

In [ ]:
modified_dict = copy(final_dict)
modified_dict['h3_extra key'] = 'extra value'
article_1 = convert_to_markdown(final_dict)
article_2 = convert_to_markdown(modified_dict)

short_1 = article_1[:1000]
short_2 = article_2[:1000]

In [ ]:
num_tokens_from_string(short_2)

In [ ]:
evaluate.logging.set_verbosity_info()
# Mauve has to compare the actual text strings themselves, not embeddings
mauve = load('mauve')
mauve_results = mauve.compute(
    predictions=[short_1],
    references=[short_2],
    device_id=0,
)
mauve_results.mauve

In [ ]:
mauve_results.mauve

In [ ]:
rouge = load('rouge')
results = rouge.compute(predictions=[short_1], references=[short_2], rouge_types=['rougeL'])
print(results['rougeL'])

In [ ]:
a = embed_ada.embed_query('hello')
a_ = str(a)
print(num_tokens_from_string(a_))

In [ ]:
for heading, content in final_dict.items():
    print(heading, num_tokens_from_string(content))

In [ ]:
final_dict['h2 See also']

In [ ]:
embed_e5.embed_query('query:  ')

In [ ]:
# Create json plan

final_headings = list(final_dict.keys())
final_content = list(final_dict.values())

def create_section_json(heading, content, id=1):
    result = {
        'section_id': id,
        'section': heading,
        'content': content,
        'section_embedding_1': embed_ada.embed_query(heading),
        'section_embedding_2': embed_e5.embed_query("query: " + heading),
        'content_embedding_1': embed_ada.embed_query(content),
        'content_embedding_2': embed_e5.embed_query("query: " + content),
    }
    print(f"Created embeddings for {heading}")
    return result

def calcuate_plan_embedding(plan: List[dict], i: int) -> List[float]:
    """Calculate plan embedding by averaging the section embeddings and content embeddings
    sequentially.

    Parameters
    ----------
    plan : List[dict]
        List of dictionaries, each containing the section and content embeddings.
    i : int
        The index of the embedding to use.
    """
    try:
        s1_mean = np.mean([x[f'section_embedding_{i}'] for x in plan], axis=0)
        c1_mean = np.mean([x[f'content_embedding_{i}'] for x in plan], axis=0)
    except KeyError:
        raise KeyError(
            f'Could not find section_embedding_{i} or content_embedding_{i} in '
            f'every element in plan. Please check that every element has the correct keys.'
        )
    total_mean = np.mean([c1_mean, s1_mean], axis=0)
    total_mean = list(total_mean)
    print(f"Calculated plan embedding {i}")
    return total_mean

# title is h1_example
title = final_headings[0][3:]
abstract = final_content[0]
plan = [
    create_section_json(heading, content, id=i)
    for i, (heading, content) in enumerate(zip(
        final_headings[1:], final_content[1:]), start=1
    )
    if 0 < num_tokens_from_string(content) <= 8000
]
plan_embed_1 = calcuate_plan_embedding(plan, 1)
plan_embed_2 = calcuate_plan_embedding(plan, 2)
plan_json = {
    'id': str(uuid4()),
    'title': title,
    'abstract': abstract,
    'title_embedding_1': embed_ada.embed_query(title),
    'title_embedding_2': embed_e5.embed_query("query: " + title),
    'abstract_embedding_1': embed_ada.embed_query(abstract),
    'abstract_embedding_2': embed_e5.embed_query("query: " + abstract),
    'plan': plan,
    'plan_embedding_1': plan_embed_1,
    'plan_embedding_2': plan_embed_2,
    'embedding1_model': 'text-embedding-ada-002',
    'embedding2_model': 'e5-base-v2',
    'success': True,
    'error': None,
}
plan_json

In [ ]:
'plan' in

In [ ]:
plan_json['plan'][0].keys()

In [ ]:
async def get_embeddings(
    input: str | List[str], model: str = "text-embedding-ada-002"
) -> List[float] | List[List[float]]:
    """This function takes one string or a list of strings and a model name,
    generates an embedding for each string in the list using the specified model,
    and returns a list of embeddings, each represented as a list of floating point
    numbers.

    Parameters
    ----------
    input : str | List[str]
        The input string or list of strings to be embedded.
    model : str, optional ['text-embedding-ada-002', 'e5-base-v2']
        The name of the model to be used for embedding.
    """
    if model == "text-embedding-ada-002":
        embedder = OpenAIEmbeddings(model=model)
    elif model == "e5-base-v2":
        embedder = HuggingFaceEmbeddings(
            model_name=f"intfloat/{model}", encode_kwargs={"normalize_embeddings": True}
        )
    else:
        raise ValueError(
            f"Model name must be 'text-embedding-ada-002' or 'e5-base-v2'. Received {model}"
        )

    if isinstance(input, str):
        try:
            return await embedder.aembed_query(input)
        except NotImplementedError as e:
            return embedder.embed_query(input)
    elif isinstance(input, list):
        try:
            return await embedder.aembed_documents(input)
        except NotImplementedError as e:
            return embedder.embed_documents(input)
    else:
        raise ValueError(
            f"Input must be a string or a list of strings. Received {type(input)}"
        )

In [ ]:
await get_embeddings(['hello', 'there'])

In [ ]:
prediction = deepcopy(plan_json)
document = deepcopy(plan_json)

def compare_documents(document: dict, prediction: dict, compare_on: str ='section'):
    """Compare the 'compare_on' sections of document and prediction. Calculate MAUVE,
    and ROUGE-L scores on the actual text, and cosine similarity on the embeddings.
    """
    if compare_on not in ['section', 'content']:
        raise ValueError(
            f"`compare_on` must be 'section' or 'content'. Received {compare_on}"
        )

    if not isinstance(document, dict) or not isinstance(prediction, dict):
        raise TypeError(
            'Both `document` and `prediction` must be dictionaries. Received '
            f'{type(document)} and {type(prediction)}'
        )

    if 'plan' not in document or 'plan' not in prediction:
        raise ValueError(
            f'Both `document` and `prediction` must contain the key "plan". At least '
            f'one of them does not.'
        )

    mauve = load("mauve")
    rouge = load("rouge")

    section_results = []
    predict_plan = prediction['plan']
    doc_plan = document['plan']
    for i, (p_dict, d_dict) in enumerate(zip(predict_plan, doc_plan), start=1):
        idx = i
        # Compute MAUVE
        mauve_results = mauve.compute(
            predictions=[p_dict['section']], references=[d_dict['section']]
        )
        mauve_score = mauve_results.mauve
        # Compute ROUGE-L
        results = rouge.compute(
            predictions=[p_dict['section']],
            references=[d_dict['section']],
            rouge_types=["rougeL"],
        )
        rouge_score = results["rougeL"]
        # Compute cosine distance between both section embeddings
        cosine_1 = cosine_similarity(
            [p_dict["section_embedding_1"]], [d_dict["section_embedding_1"]]
        )[0][0]
        cosine_2 = cosine_similarity(
            [p_dict["section_embedding_2"]], [d_dict["section_embedding_2"]]
        )[0][0]
        # Combine results
        result = {
            'section_id': idx,
            'mauve_similarity': mauve_score,
            'rouge_L_similarity': rouge_score,
            'embedding1_cosine_similarity': cosine_1,
            'embedding2_cosine_similarity': cosine_2,
        }
        section_results.append(result)

    # Calcualte total scores
    mauve_total = np.mean([x['mauve_similarity'] for x in section_results])
    rouge_total = np.mean([x['rouge_L_similarity'] for x in section_results])
    cosine_1_total = np.mean([x['embedding1_cosine_similarity'] for x in section_results])
    cosine_2_total = np.mean([x['embedding2_cosine_similarity'] for x in section_results])

    total_results = {
        'mauve_similarity': mauve_total,
        'rouge_L_similarity': rouge_total,
        'embedding1_cosine_similarity': cosine_1_total,
        'embedding2_cosine_similarity': cosine_2_total,
    }

    if compare_on == 'section':
        compare_on = 'plan'

    output = {
        'document_id': document['id'],
        'prediction_id': prediction['id'],
        f'{compare_on}_total_similarity': total_results,
        f'{compare_on}_bysection_similarity': section_results,
    }
    return output

In [ ]:
sec_results = compare_documents(document, prediction, compare_on='section')

In [ ]:
content_results = compare_documents(document, prediction, compare_on='content')

In [ ]:
sec_results

In [ ]:
section_results

In [8]:
total_results

NameError: name 'total_results' is not defined

In [ ]:
num_tokens_from_string('')

In [ ]:
def create_section_json(heading, content, id=1):
    return {
        'section_id': id,
        'section': heading,
        'content': content,
        'section_embedding_1': embed_ada.embed_query(heading),
        'section_embedding_2': embed_e5.embed_query("query: " + heading),
        'content_embedding_1': embed_ada.embed_query(content),
        'content_embedding_2': embed_e5.embed_query("query: " + content),
    }

# plan_embedding_1 = np.mean([s['section_embedding_1']])
plan = [
    dict(s1=[1,2,3], c1=[4,5,6]),
    dict(s1=[7,8,9], c1=[10,11,12]),
]
s1_mean = np.mean([x['s1'] for x in plan], axis=0)
c1_mean = np.mean([x['c1'] for x in plan], axis=0)
total_mean = np.mean([c1_mean, s1_mean], axis=0)
total_mean

In [ ]:
(26)/4

In [ ]:
def compare_content(plan_1: List[dict], plan_2: List[dict]):
    document_id = uuid4()
    prediction_id = uuid4()
    total_similarity = 'not possible'
    by_section_similarity = ...

In [ ]:
def compare_plans(plan_1, plan_2):
    # Mauve has to compare the actual text strings themselves, not embeddings
    mauve = load('mauve')
    mauve_results = mauve.compute(predictions=[article_1], references=[article_2])
    mauve_results.mauve

    cosine_1 = cosine_similarity([plan_1['plan_embedding_1']], [plan_2['plan_embedding_1']])[0][0]
    cosine_2 = cosine_similarity([plan_1['plan_embedding_2']], [plan_2['plan_embedding_2']])[0][0]
    mauve_1 = ...
    return {
        'document_id': 'what is this?',
        'prediction_id': 'what is this?',
        'plan_similarity': '...'
    }

In [ ]:
ex = [{"section_id": 1, "section": "Introduction", "content": "Introduction content...", "section_embedding1": [0.1, 0.2, 0.3]}]
embed_ada.embed_query(str(ex))
embed_e5.embed_query("query: " + str(ex))

In [ ]:
id(12)

# Unused Code

In [ ]:
def convert_to_markdown(article_dict):
    md_text = ""

    for heading, content in article_dict.items():
        # heading is of form: 'h3 Example'
        # Define the markdown equivalent for the heading level
        heading_level = "#" * int(heading[1])
        heading = heading[3:]
        # Append the heading and the content to the markdown text
        md_text += f"{heading_level} {heading}\n\n{content}\n\n"

    return md_text

article_dict = {
    'h1_Self-driving car': 'Content for Self-driving car',
    'h2_History': 'Content for History',
    'h2_Definitions': 'Content for Definitions',
    'h3_Terminology and safety considerations': 'Content for Terminology and safety considerations',
    'h3_Autonomous vs. automa': 'Content for Autonomous vs. automa',
    'h3_Autonomous versus cooperativ': 'Content for Autonomous versus cooperativ',
    'h2_Classifications': 'Content for Classifications',
    'h3_Self-driving car': 'Content for Self-driving car',
    'h3_SAE classification': 'Content for SAE classification',
    'h3_Levels of driving automation': 'Content for Levels of driving automation'
}

self_driving_md = convert_to_markdown(doc_dict)

In [ ]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("####", "Header 4"),
    ("#####", "Header 5"),
    ("######", "Header 6"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
# markdown_splitter = MarkdownTextSplitter(chunk_size=512, chunk_overlap=0)
docs = markdown_splitter.split_text(self_driving_md)
# docs[0]Document(page_content="Dual phase evolution (DPE) is a process that drives self-organization within complex adaptive systems.[1] It arises in response to phase changes within the network of connections formed by a system's components. DPE occurs in a wide range of physical, biological and social systems. Its applications to technology include methods for manufacturing novel materials and algorithms to solve complex problems in computation.", metadata={'Header 1': 'Dual-phase evolution'})
docs[1]

In [ ]:
embed_e5 = SentenceTransformer('intfloat/e5-base-v2')

In [ ]:
"query: " + final_docs[0].page_content

In [ ]:
embeddings = embed_e5.encode("query: " + final_docs[0].page_content, normalize_embeddings=True)
embeddings

In [ ]:
embedding_openai = openai.Embedding.create(
    input = [final_docs[0].page_content],
    model="text-embedding-ada-002")
['data'][0]['embedding']
embedding_openai

In [ ]:
for i, doc in enumerate(docs):
    tokens = num_tokens_from_string(doc.page_content)
    if tokens > 512:
        print(f"Document {i}")
        print(doc.metadata)
        print(f"{num_tokens_from_string(doc.page_content):,} tokens")
        print(doc.page_content[:100])
        print()

In [ ]:
history = docs[1]
history

In [ ]:
final_docs: List[Document] = []
start_docs = copy(docs)
for i, doc in enumerate(start_docs):
    num_tokens = num_tokens_from_string(doc.page_content)
    max_allowed_tokens = 512
    if num_tokens <= max_allowed_tokens:
        # Add topics
        topics = await extract_topics(doc.page_content)
        doc.metadata['topics'] = topics
        final_docs.append(doc)
    else:
        # Split the document into smaller chunks, then add topics
        char_splitter = RecursiveCharacterTextSplitter(
            chunk_size=max_allowed_tokens,
            chunk_overlap=0,
            # ' ' separator means sometimes sentences will be cut in two to ensure
            # the chunk size is not exceeded
            separators=["\n\n", "\n", ' '],
            length_function=num_tokens_from_string,
        )
        splits: List[str] = char_splitter.split_text(doc.page_content)
        for split in splits:
            new_metadata = doc.metadata.copy()
            topics = await extract_topics(split)
            new_metadata['topics'] = topics
            final_docs.append(Document(page_content=split, metadata=new_metadata))

print(f"We started with {len(start_docs)} docs and got {len(final_docs)} final docs")
print(f"That's a {len(final_docs) / len(start_docs):.2f}x increase in docs")
for d in final_docs:
    print(d.metadata)
    print(f"{num_tokens_from_string(d.page_content):,} tokens")
    print(d.page_content[:100])
    print()

In [ ]:
char_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=0,
    separators=["\n\n", "\n"],
    length_function=num_tokens_from_string,
)
splits = char_splitter.split_text(history.page_content)
print(f"We got {len(splits)} splits")

for i, split in enumerate(splits):
    topics = await extract_topics(split)
    print(f"Split {i} - Topics: {topics}")
    print(f"{num_tokens_from_string(split):,} tokens, {len(split)} characters")
    print(split)
    print()

In [ ]:
splits

In [ ]:
history.metadata['example'] = 'hello'
history.metadata

In [ ]:
base_metadata = history.metadata.copy()
base_metadata['topics'] = topics
Document(page_content=split, metadata=base_metadata)

In [ ]:
class MarkdownHeaderTextSplitterMaxTokenLimit(MarkdownHeaderTextSplitter):
    def __init__(self, max_tokens=512, **kwargs):
        super().__init__(**kwargs)
        self.max_tokens = max_tokens

    def calculate_tokens(self, string: str, encoding_name: str = "gpt-3.5-turbo") -> int:
        """Returns the number of tokens in a text string."""
        try:
            encoding = tiktoken.get_encoding(encoding_name)
        except ValueError:
            encoding = tiktoken.encoding_for_model(encoding_name)
        num_tokens = len(encoding.encode(string))
        return num_tokens

    def generate_extra_metadata(self, string: str):
        return 'EXTRA INFO ADDED'

    def aggregate_lines_to_chunks(self, lines: List[LineType]) -> List[Document]:
        """Combine lines with common metadata into chunks
        Args:
            lines: Line of text / associated header metadata
        """
        aggregated_chunks: List[LineType] = []

        for line in lines:
            if (
                aggregated_chunks
                and aggregated_chunks[-1]["metadata"] == line["metadata"]
            ):
                # If the last line in the aggregated list
                # has the same metadata as the current line,
                # append the current content to the last lines's content
                new_content = aggregated_chunks[-1]["content"] + "  \n" + line["content"]
                new_token_count = self.calculate_tokens(new_content)

                # If less than 512 tokens, update the last line's content
                if new_token_count <= 512:
                    aggregated_chunks[-1]["content"] = new_content
                # If more than 512 tokens, start a new chunk and add extra metadata to it
                else:
                    # Generate the extra metadata for the current chunk.
                    new_metadata = self.generate_extra_metadata(line["content"])
                    line["metadata"] += new_metadata
                    # If the new token count would exceed the limit, start a new chunk
                    aggregated_chunks.append(line)
            else:
                # Otherwise, append the current line to the aggregated list
                aggregated_chunks.append(line)

        return [
            Document(page_content=chunk["content"], metadata=chunk["metadata"])
            for chunk in aggregated_chunks
        ]


In [ ]:

import urllib.request
from urllib.parse import quote

def download_html(url, output_file):
    response = urllib.request.urlopen(url)
    data = response.read()

    with open(output_file, 'wb') as file:
        file.write(data)

def convert_html_to_markdown(input_file, output_file):
    command = ['pandoc', '-f', 'html', '-t', 'markdown', '-o', output_file, input_file]
    subprocess.run(command, check=True)

# Prepare the URL
wiki_page = 'https://en.wikipedia.org/wiki/OpenAI'
wiki_page_encoded = quote(wiki_page, safe=':/')
html_file = 'openai.html'
markdown_file = 'openai.md'

# Download the HTML page
download_html(wiki_page_encoded, html_file)

# Convert HTML to Markdown
convert_html_to_markdown(html_file, markdown_file)

print(f"Markdown file {markdown_file} has been created.")


In [ ]:
with open('openai.md', 'r', encoding='utf-8') as file:
    markdown_text = file.read()

markdown_splitter = MarkdownTextSplitter(chunk_size=512, chunk_overlap=0)
docs = markdown_splitter.create_documents([markdown_text])
docs[1]

In [ ]:
import subprocess
import urllib.request
from urllib.parse import quote
from bs4 import BeautifulSoup

def download_html(url):
    response = urllib.request.urlopen(url)
    data = response.read().decode()
    return data

def parse_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    content_text = soup.find('div', {'id': 'mw-content-text'})
    return str(content_text)

def convert_html_to_markdown(html_content, output_file):
    with open('temp.html', 'w') as f:
        f.write(html_content)
    command = ['pandoc', '-f', 'html', '-t', 'markdown', '-o', output_file, 'temp.html']
    subprocess.run(command, check=True)

# Prepare the URL
wiki_page = 'https://en.wikipedia.org/wiki/OpenAI'
wiki_page_encoded = quote(wiki_page, safe=':/')
markdown_file = 'openai.md'

# Download the HTML page
html_data = download_html(wiki_page_encoded)

# Parse HTML
parsed_html = parse_html(html_data)

# Convert HTML to Markdown
convert_html_to_markdown(parsed_html, markdown_file)

print(f"Markdown file {markdown_file} has been created.")


In [ ]:
llm = OpenAI()

wikipedia_articles = [
    "https://en.wikipedia.org/wiki/Large_language_model",
    "https://en.wikipedia.org/wiki/Transformer_(machine_learning_model)",
    "https://en.wikipedia.org/wiki/Dual-phase_evolution",
    'https://en.wikipedia.org/wiki/Simulated_annealing',
    "https://en.wikipedia.org/wiki/Tessellation",
    "https://en.wikipedia.org/wiki/Climate_change",
    "https://en.wikipedia.org/wiki/DNA_nanotechnology",
    "https://en.wikipedia.org/wiki/Self-driving_car",
    "https://en.wikipedia.org/wiki/Unmanned_aerial_vehicle",
    "https://en.wikipedia.org/wiki/2022%E2%80%932023_food_crises",
    "https://en.wikipedia.org/wiki/Economic_impacts_of_climate_change",
]

dual_phase = wikipedia_articles[2]

article = wikipedia_articles[0]

# Send an HTTP request to the URL
response = requests.get(article)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Get the HTML content from the response
    html_content = response.text

    # Use BeautifulSoup to parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
else:
    print(f"Failed to download the webpage. Status code: {response.status_code}")

In [ ]:
all_articles = WebBaseLoader(wikipedia_articles).load()
print('Total number of articles: ', len(all_articles))
for i, doc in enumerate(all_articles):
    print(f"Article {i} contains {num_tokens_from_string(doc.page_content):,} tokens - {wikipedia_articles[i]}")

In [ ]:
loader = WebBaseLoader(article)
data = loader.load()
len(data)

In [ ]:
dual_phase = 'https://en.wikipedia.org/wiki/Dual-phase_evolution'
# dual_phase = 'https://en.wikipedia.org/wiki/Climate_change'
response_dp = requests.get(dual_phase)
html_content_dp = response_dp.text
soup_dp = BeautifulSoup(html_content_dp, 'html.parser')

# Extract all headers and titles
titles_and_headers = []

# Find all title tags and extract their text
title_tags = soup_dp.find_all('title')
for title in title_tags:
    titles_and_headers.append(title.text.strip())

# Find all header tags (h1 to h6) and extract their text
header_tags = soup_dp.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
for header in header_tags:
    titles_and_headers.append(header.text.strip())

# Print the extracted headers and titles
for item in titles_and_headers:
    print(item)


In [ ]:
soup_dp.h1

In [ ]:
with open('self_driving.html', 'w', encoding='utf-8') as file:
    file.write(soup.prettify())

In [ ]:
[x for x in soup.find_all('h3') if 'Approaches' in x.get_text()][0].find_next()

In [ ]:
tags = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']
found_tags = soup.find_all(tags)

# Extract tags and their associated content into a dictionary
found_tags[1].name

In [ ]:
# Assuming 'my_dict' is your dictionary
my_dict = {
    'Contents': None,
    'Dual-phase evolution': None,
    'Introduction[edit]': None,
    'The DPE mechanism[edit]': None,
    'Underlying network[edit]': None,
    'Phase shifts[edit]': None,
    'Selection and variation[edit]': None
}

# Remove '[edit]' from the end of the keys
for key in list(my_dict.keys()):  # Using list() to avoid changing the dictionary size during iteration
    new_key = key.rstrip('[edit]')
    if new_key != key:
        my_dict[new_key] = my_dict.pop(key)

print(my_dict)


In [ ]:
soup.h1.find_next_sibling().find_next_sibling().find_next_sibling()

In [ ]:
# Load the webpage content
url = 'https://en.wikipedia.org/wiki/Dual-phase_evolution'
r = requests.get(url)
html_content = r.text

# Create a BeautifulSoup object
soup = BeautifulSoup(html_content, 'html.parser')

# Remove unwanted tags: script, style, [document], head, title
for element in soup(["script", "style", "head", "title", "[document]"]):
    element.decompose()

# Also remove HTML comments
for element in soup.find_all(string=lambda text: isinstance(text, Comment)):
    element.extract()

# Define the tags to find
tags = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']
found_tags = soup.find_all(tags)

# Extract tags and their associated content into a dictionary
doc_dict = {}
for tag in found_tags:
    content = []
    next_sibling = tag.find_next_sibling('p')

    while next_sibling and next_sibling.name not in tags:
        if next_sibling.name == 'p':
            content.append(next_sibling.get_text(strip=True))
        next_sibling = next_sibling.find_next_sibling()

    doc_dict[tag.get_text(strip=True)] = " ".join(content)

pprint(doc_dict, sort_dicts=False)

In [ ]:
with open('soup.html', 'w', encoding='utf-8') as file:
    file.write(soup.prettify())

In [ ]:
broken_html = """<h1 class="firstHeading mw-first-heading" id="firstHeading">
        <span class="mw-page-title-main">
         Dual-phase evolution
        </span>
       </h1>
       <div class="vector-dropdown mw-portlet mw-portlet-lang" id="p-lang-btn">
        <input aria-haspopup="true" aria-label="Go to an article in another language. Available in 1 language" class="vector-dropdown-checkbox mw-interlanguage-selector" data-event-name="ui.dropdown-p-lang-btn" id="p-lang-btn-checkbox" role="button" type="checkbox">
         <label aria-hidden="true" class="vector-dropdown-label cdx-button cdx-button--fake-button cdx-button--fake-button--enabled cdx-button--weight-quiet cdx-button--action-progressive mw-portlet-lang-heading-1" for="p-lang-btn-checkbox" id="p-lang-btn-label">
         </label>
         <span class="vector-icon mw-ui-icon-language-progressive mw-ui-icon-wikimedia-language-progressive">
         </span>
         <span class="vector-dropdown-label-text">
          1 language
         </span>
         <div class="vector-dropdown-content">
          <div class="vector-menu-content">
           <ul class="vector-menu-content-list">
            <li class="interlanguage-link interwiki-fa mw-list-item">
             <a class="interlanguage-link-target" href="https://fa.wikipedia.org/wiki/%D9%81%D8%B1%DA%AF%D8%B4%D8%AA_%D8%AF%D9%88%D9%81%D8%A7%D8%B2%DB%8C" hreflang="fa" lang="fa" title="فرگشت دوفازی – Persian">
              <span>
               فارسی
              </span>
             </a>
            </li>
           </ul>
           <div class="after-portlet after-portlet-lang">
            <span class="wb-langlinks-edit wb-langlinks-link">
             <a class="wbc-editpage" href="https://www.wikidata.org/wiki/Special:EntityPage/Q25109659#sitelinks-wikipedia" title="Edit interlanguage links">
              Edit links
             </a>
            </span>
           </div>
          </div>
         </div>
        </input>
       </div>
      </header>
      <div class="vector-page-toolbar">
       <div class="vector-page-toolbar-container">
        <div id="left-navigation">
         <nav aria-label="Namespaces">
          <div class="vector-menu vector-menu-tabs mw-portlet mw-portlet-associated-pages" id="p-associated-pages">
           <div class="vector-menu-content">
            <ul class="vector-menu-content-list">
             <li class="selected vector-tab-noicon mw-list-item" id="ca-nstab-main">
              <a accesskey="c" class="" data-mw="interface" href="/wiki/Dual-phase_evolution" title="View the content page [c]">
               <span>
                Article
               </span>
              </a>
             </li>
             <li class="vector-tab-noicon mw-list-item" id="ca-talk">
              <a accesskey="t" class="" data-mw="interface" href="/wiki/Talk:Dual-phase_evolution" rel="discussion" title="Discuss improvements to the content page [t]">
               <span>
                Talk
               </span>
              </a>
             </li>
            </ul>
           </div>
          </div>
          <div class="vector-dropdown mw-portlet mw-portlet-variants emptyPortlet" id="p-variants">
           <input aria-haspopup="true" aria-label="Change language variant" class="vector-dropdown-checkbox" data-event-name="ui.dropdown-p-variants" id="p-variants-checkbox" role="button" type="checkbox"/>
           <label aria-hidden="true" class="vector-dropdown-label" for="p-variants-checkbox" id="p-variants-label">
           </label>
           <span class="vector-dropdown-label-text">
            English
           </span>
           <div class="vector-dropdown-content">
            <div class="vector-menu-content">
             <ul class="vector-menu-content-list">
             </ul>
            </div>
           </div>
          </div>
         </nav>
        </div>
        <div class="vector-collapsible" id="right-navigation">
         <nav aria-label="Views">
          <div class="vector-menu vector-menu-tabs mw-portlet mw-portlet-views" id="p-views">
           <div class="vector-menu-content">
            <ul class="vector-menu-content-list">
             <li class="selected vector-tab-noicon mw-list-item" id="ca-view">
              <a class="" data-mw="interface" href="/wiki/Dual-phase_evolution">
               <span>
                Read
               </span>
              </a>
             </li>
             <li class="vector-tab-noicon mw-list-item" id="ca-edit">
              <a accesskey="e" class="" data-mw="interface" href="/w/index.php?title=Dual-phase_evolution&amp;action=edit" title="Edit this page [e]">
               <span>
                Edit
               </span>
              </a>
             </li>
             <li class="vector-tab-noicon mw-list-item" id="ca-history">
              <a accesskey="h" class="" data-mw="interface" href="/w/index.php?title=Dual-phase_evolution&amp;action=history" title="Past revisions of this page [h]">
               <span>
                View history
               </span>
              </a>
             </li>
            </ul>
           </div>
          </div>
         </nav>
         <nav aria-label="More options" class="vector-page-tools-landmark">
          <div class="vector-dropdown vector-page-tools-dropdown" id="vector-page-tools-dropdown">
           <input aria-haspopup="true" aria-label="Tools" class="vector-dropdown-checkbox" data-event-name="ui.dropdown-vector-page-tools-dropdown" id="vector-page-tools-dropdown-checkbox" role="button" type="checkbox"/>
           <label aria-hidden="true" class="vector-dropdown-label cdx-button cdx-button--fake-button cdx-button--fake-button--enabled cdx-button--weight-quiet" for="vector-page-tools-dropdown-checkbox" id="vector-page-tools-dropdown-label">
           </label>
           <span class="vector-dropdown-label-text">
            Tools
           </span>
           <div class="vector-dropdown-content">
            <div class="vector-unpinned-container" id="vector-page-tools-unpinned-container">
             <div class="vector-page-tools vector-pinnable-element" id="vector-page-tools">
              <div class="vector-pinnable-header vector-page-tools-pinnable-header vector-pinnable-header-unpinned" data-feature-name="page-tools-pinned" data-pinnable-element-id="vector-page-tools" data-pinned-container-id="vector-page-tools-pinned-container" data-unpinned-container-id="vector-page-tools-unpinned-container">
               <div class="vector-pinnable-header-label">
                Tools
               </div>
               <button class="vector-pinnable-header-toggle-button vector-pinnable-header-pin-button" data-event-name="pinnable-header.vector-page-tools.pin">
                move to sidebar
               </button>
               <button class="vector-pinnable-header-toggle-button vector-pinnable-header-unpin-button" data-event-name="pinnable-header.vector-page-tools.unpin">
                hide
               </button>
              </div>
              <div class="vector-menu mw-portlet mw-portlet-cactions emptyPortlet vector-has-collapsible-items" id="p-cactions" title="More options">
               <div class="vector-menu-heading">
                Actions
               </div>
               <div class="vector-menu-content">
                <ul class="vector-menu-content-list">
                 <li class="selected vector-more-collapsible-item mw-list-item" id="ca-more-view">
                  <a href="/wiki/Dual-phase_evolution">
                   <span>
                    Read
                   </span>
                  </a>
                 </li>
                 <li class="vector-more-collapsible-item mw-list-item" id="ca-more-edit">
                  <a href="/w/index.php?title=Dual-phase_evolution&amp;action=edit">
                   <span>
                    Edit
                   </span>
                  </a>
                 </li>
                 <li class="vector-more-collapsible-item mw-list-item" id="ca-more-history">
                  <a href="/w/index.php?title=Dual-phase_evolution&amp;action=history">
                   <span>
                    View history
                   </span>
                  </a>
                 </li>
                </ul>
               </div>
              </div>
              <div class="vector-menu mw-portlet mw-portlet-tb" id="p-tb">
               <div class="vector-menu-heading">
                General
               </div>
               <div class="vector-menu-content">
                <ul class="vector-menu-content-list">
                 <li class="mw-list-item" id="t-whatlinkshere">
                  <a accesskey="j" href="/wiki/Special:WhatLinksHere/Dual-phase_evolution" title="List of all English Wikipedia pages containing links to this page [j]">
                   <span>
                    What links here
                   </span>
                  </a>
                 </li>
                 <li class="mw-list-item" id="t-recentchangeslinked">
                  <a accesskey="k" href="/wiki/Special:RecentChangesLinked/Dual-phase_evolution" rel="nofollow" title="Recent changes in pages linked from this page [k]">
                   <span>
                    Related changes
                   </span>
                  </a>
                 </li>
                 <li class="mw-list-item" id="t-upload">
                  <a accesskey="u" href="/wiki/Wikipedia:File_Upload_Wizard" title="Upload files [u]">
                   <span>
                    Upload file
                   </span>
                  </a>
                 </li>
                 <li class="mw-list-item" id="t-specialpages">
                  <a accesskey="q" href="/wiki/Special:SpecialPages" title="A list of all special pages [q]">
                   <span>
                    Special pages
                   </span>
                  </a>
                 </li>
                 <li class="mw-list-item" id="t-permalink">
                  <a href="/w/index.php?title=Dual-phase_evolution&amp;oldid=1126289474" title="Permanent link to this revision of this page">
                   <span>
                    Permanent link
                   </span>
                  </a>
                 </li>
                 <li class="mw-list-item" id="t-info">
                  <a href="/w/index.php?title=Dual-phase_evolution&amp;action=info" title="More information about this page">
                   <span>
                    Page information
                   </span>
                  </a>
                 </li>
                 <li class="mw-list-item" id="t-cite">
                  <a href="/w/index.php?title=Special:CiteThisPage&amp;page=Dual-phase_evolution&amp;id=1126289474&amp;wpFormIdentifier=titleform" title="Information on how to cite this page">
                   <span>
                    Cite this page
                   </span>
                  </a>
                 </li>
                 <li class="mw-list-item" id="t-wikibase">
                  <a accesskey="g" href="https://www.wikidata.org/wiki/Special:EntityPage/Q25109659" title="Structured data on this page hosted by Wikidata [g]">
                   <span>
                    Wikidata item
                   </span>
                  </a>
                 </li>
                </ul>
               </div>
              </div>
              <div class="vector-menu mw-portlet mw-portlet-coll-print_export" id="p-coll-print_export">
               <div class="vector-menu-heading">
                Print/export
               </div>
               <div class="vector-menu-content">
                <ul class="vector-menu-content-list">
                 <li class="mw-list-item" id="coll-download-as-rl">
                  <a href="/w/index.php?title=Special:DownloadAsPdf&amp;page=Dual-phase_evolution&amp;action=show-download-screen" title="Download this page as a PDF file">
                   <span>
                    Download as PDF
                   </span>
                  </a>
                 </li>
                 <li class="mw-list-item" id="t-print">
                  <a accesskey="p" href="/w/index.php?title=Dual-phase_evolution&amp;printable=yes" title="Printable version of this page [p]">
                   <span>
                    Printable version
                   </span>
                  </a>
                 </li>
                </ul>
               </div>
              </div>
             </div>
            </div>
           </div>
          </div>
         </nav>
        </div>
       </div>
      </div>
      <div class="vector-column-end">
       <nav aria-label="More options" class="vector-page-tools-landmark vector-sticky-pinned-container">
        <div class="vector-pinned-container" id="vector-page-tools-pinned-container">
        </div>
       </nav>
      </div>
      <div aria-labelledby="firstHeading" class="vector-body" data-mw-ve-target-container="" id="bodyContent">
       <div class="vector-body-before-content">
        <div class="mw-indicators">
        </div>
        <div class="noprint" id="siteSub">
         From Wikipedia, the free encyclopedia
        </div>
       </div>
       <div id="contentSub">
        <div id="mw-content-subtitle">
        </div>
       </div>
       <div class="mw-body-content mw-content-ltr" dir="ltr" id="mw-content-text" lang="en">
        <div class="mw-parser-output">
         <div class="shortdescription nomobile noexcerpt noprint searchaux" style="display:none">
          Process that drives self-organization within complex adaptive systems
         </div>
         <table class="box-Multiple_issues plainlinks metadata ambox ambox-content ambox-multiple_issues compact-ambox" role="presentation">
          <tbody>
           <tr>
            <td class="mbox-image">
             <div class="mbox-image-div">
              <img alt="" data-file-height="40" data-file-width="40" decoding="async" height="40" src="//upload.wikimedia.org/wikipedia/en/thumb/b/b4/Ambox_important.svg/40px-Ambox_important.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/b/b4/Ambox_important.svg/60px-Ambox_important.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/b/b4/Ambox_important.svg/80px-Ambox_important.svg.png 2x" width="40"/>
             </div>
            </td>
            <td class="mbox-text">
             <div class="mbox-text-span">
              <div class="multiple-issues-text mw-collapsible">
               <b>
                This article has multiple issues.
               </b>
               Please help
               <b>
                <a class="external text" href="https://en.wikipedia.org/w/index.php?title=Dual-phase_evolution&amp;action=edit">
                 improve it
                </a>
               </b>
               or discuss these issues on the
               <b>
                <a href="/wiki/Talk:Dual-phase_evolution" title="Talk:Dual-phase evolution">
                 talk page
                </a>
               </b>
               .
               <small>
                <i>
                 (
                 <a href="/wiki/Help:Maintenance_template_removal" title="Help:Maintenance template removal">
                  Learn how and when to remove these template messages
                 </a>
                 )
                </i>
               </small>
               <div class="mw-collapsible-content">
                <link href="mw-data:TemplateStyles:r1097763485" rel="mw-deduplicated-inline-style"/>
                <table class="box-More_citations_needed plainlinks metadata ambox ambox-content ambox-Refimprove" role="presentation">
                 <tbody>
                  <tr>
                   <td class="mbox-image">
                    <div class="mbox-image-div">
                     <a class="image" href="/wiki/File:Question_book-new.svg">
                      <img alt="" data-file-height="399" data-file-width="512" decoding="async" height="39" src="//upload.wikimedia.org/wikipedia/en/thumb/9/99/Question_book-new.svg/50px-Question_book-new.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/9/99/Question_book-new.svg/75px-Question_book-new.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/9/99/Question_book-new.svg/100px-Question_book-new.svg.png 2x" width="50"/>
                     </a>
                    </div>
                   </td>
                   <td class="mbox-text">
                    <div class="mbox-text-span">
                     This article
                     <b>
                      needs additional citations for
                      <a href="/wiki/Wikipedia:Verifiability" title="Wikipedia:Verifiability">
                       verification
                      </a>
                     </b>
                     .
                     <span class="hide-when-compact">
                      Please help
                      <a class="external text" href="https://en.wikipedia.org/w/index.php?title=Dual-phase_evolution&amp;action=edit">
                       improve this article
                      </a>
                      by
                      <a href="/wiki/Help:Referencing_for_beginners" title="Help:Referencing for beginners">
                       adding citations to reliable sources
                      </a>
                      . Unsourced material may be challenged and removed.
                      <br/>
                      <small>
                       <span class="plainlinks">
                        <i>
                         Find sources:
                        </i>
                        <a class="external text" href="https://www.google.com/search?as_eq=wikipedia&amp;q=%22Dual-phase+evolution%22" rel="nofollow">
                         "Dual-phase evolution"
                        </a>
                        –
                        <a class="external text" href="https://www.google.com/search?tbm=nws&amp;q=%22Dual-phase+evolution%22+-wikipedia&amp;tbs=ar:1" rel="nofollow">
                         news
                        </a>
                        <b>
                         ·
                        </b>
                        <a class="external text" href="https://www.google.com/search?&amp;q=%22Dual-phase+evolution%22&amp;tbs=bkt:s&amp;tbm=bks" rel="nofollow">
                         newspapers
                        </a>
                        <b>
                         ·
                        </b>
                        <a class="external text" href="https://www.google.com/search?tbs=bks:1&amp;q=%22Dual-phase+evolution%22+-wikipedia" rel="nofollow">
                         books
                        </a>
                        <b>
                         ·
                        </b>
                        <a class="external text" href="https://scholar.google.com/scholar?q=%22Dual-phase+evolution%22" rel="nofollow">
                         scholar
                        </a>
                        <b>
                         ·
                        </b>
                        <a class="external text" href="https://www.jstor.org/action/doBasicSearch?Query=%22Dual-phase+evolution%22&amp;acc=on&amp;wc=on" rel="nofollow">
                         JSTOR
                        </a>
                       </span>
                      </small>
                     </span>
                     <span class="date-container">
                      <i>
                       (
                       <span class="date">
                        May 2015
                       </span>
                       )
                      </i>
                     </span>
                     <span class="hide-when-compact">
                      <i>
                       (
                       <small>
                        <a href="/wiki/Help:Maintenance_template_removal" title="Help:Maintenance template removal">
                         Learn how and when to remove this template message
                        </a>
                       </small>
                       )
                      </i>
                     </span>
                    </div>
                   </td>
                  </tr>
                 </tbody>
                </table>
                <link href="mw-data:TemplateStyles:r1097763485" rel="mw-deduplicated-inline-style"/>
                <table class="box-Technical plainlinks metadata ambox ambox-style ambox-technical" role="presentation">
                 <tbody>
                  <tr>
                   <td class="mbox-image">
                    <div class="mbox-image-div">
                     <img alt="" data-file-height="48" data-file-width="48" decoding="async" height="40" src="//upload.wikimedia.org/wikipedia/en/thumb/f/f2/Edit-clear.svg/40px-Edit-clear.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/f/f2/Edit-clear.svg/60px-Edit-clear.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/f/f2/Edit-clear.svg/80px-Edit-clear.svg.png 2x" width="40"/>
                    </div>
                   </td>
                   <td class="mbox-text">
                    <div class="mbox-text-span">
                     This article
                     <b>
                      may be too technical for most readers to understand
                     </b>
                     .
                     <span class="hide-when-compact">
                      Please
                      <a class="external text" href="https://en.wikipedia.org/w/index.php?title=Dual-phase_evolution&amp;action=edit">
                       help improve it
                      </a>
                      to
                      <a href="/wiki/Wikipedia:Make_technical_articles_understandable" title="Wikipedia:Make technical articles understandable">
                       make it understandable to non-experts
                      </a>
                      , without removing the technical details.
                     </span>
                     <span class="date-container">
                      <i>
                       (
                       <span class="date">
                        May 2015
                       </span>
                       )
                      </i>
                     </span>
                     <span class="hide-when-compact">
                      <i>
                       (
                       <small>
                        <a href="/wiki/Help:Maintenance_template_removal" title="Help:Maintenance template removal">
                         Learn how and when to remove this template message
                        </a>
                       </small>
                       )
                      </i>
                     </span>
                    </div>
                   </td>
                  </tr>
                 </tbody>
                </table>
               </div>
              </div>
              <span class="hide-when-compact">
               <i>
                (
                <small>
                 <a href="/wiki/Help:Maintenance_template_removal" title="Help:Maintenance template removal">
                  Learn how and when to remove this template message
                 </a>
                </small>
                )
               </i>
              </span>
             </div>
            </td>
           </tr>
          </tbody>
         </table>
         <p>
          <b>
           Dual phase evolution
          </b>
          (
          <b>
           DPE
          </b>
          ) is a process that drives
          <a href="/wiki/Self-organization" title="Self-organization">
           self-organization
          </a>
          within
          <a href="/wiki/Complex_adaptive_system" title="Complex adaptive system">
           complex adaptive systems
          </a>
          .
          <sup class="reference" id="cite_ref-DPE2_1-0">
           <a href="#cite_note-DPE2-1">
            [1]
           </a>
          </sup>
          It arises in response to phase changes within the network of connections formed by a system's components. DPE occurs in a wide range of physical, biological and social systems. Its applications to technology include methods for manufacturing novel materials and algorithms to solve complex problems in computation.
         </p>
         <meta property="mw:PageProp/toc">
          <h2>
           <span class="mw-headline" id="Introduction">
            Introduction
           </span>
           <span class="mw-editsection">
            <span class="mw-editsection-bracket">
             [
            </span>
            <a href="/w/index.php?title=Dual-phase_evolution&amp;action=edit&amp;section=1" title="Edit section: Introduction">
             edit
            </a>
            <span class="mw-editsection-bracket">
             ]
            </span>
           </span>
          </h2>
          <p>
           Dual phase evolution (DPE) is a process that promotes the emergence of large-scale order in
           <a class="mw-redirect" href="/wiki/Complex_systems" title="Complex systems">
            complex systems
           </a>"""

working_code = """from bs4 import BeautifulSoup, Comment
import requests

# Load the webpage content
url = 'http://your-website-url.com'
r = requests.get(url)
html_content = r.text

# Create a BeautifulSoup object
soup = BeautifulSoup(html_content, 'html.parser')

# Remove unwanted tags: script, style, [document], head, title
for element in soup(["script", "style", "head", "title", "[document]"]):
    element.decompose()

# Also remove HTML comments
for element in soup.find_all(text=lambda text: isinstance(text, Comment)):
    element.extract()

# Define the tags to find
tags = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']
found_tags = soup.find_all(tags)

# Extract tags and their associated content into a dictionary
doc_dict = {}
for tag in found_tags:
    content = []
    next_sibling = tag.find_next_sibling()

    while next_sibling and next_sibling.name not in tags:
        if next_sibling.name == 'p':
            content.append(next_sibling.get_text(strip=True))
        next_sibling = next_sibling.find_next_sibling()

    doc_dict[tag.get_text(strip=True)] = " ".join(content)

# Print the resulting dictionary
for k, v in doc_dict.items():
    print(f'{k}: {v}')
"""

prompt = f"""The following code was created by GPT-4 and successfully for each header tag finds and returns all the subsequent p tags until the next header tag and keeps doing that for each header tag and adds them to a dict.

However, it does not find the p tag for the h1 tag. Please correct it so that
it finds the p tag after the h1 tag

Working code: ####
{working_code}
####

html soup: ####
{broken_html}
"""
llm_16k.predict(prompt)

In [ ]:
pprint('To fix the code, you need to modify the while loop condition to include the h1 tag. Here\'s the modified code:\n\n```python\ndoc_dict = {}\nfor tag in found_tags:\n    content = []\n    next_sibling = tag.find_next_sibling()\n\n    while next_sibling and (next_sibling.name not in tags or tag.name == \'h1\'):\n        if next_sibling.name == \'p\':\n            content.append(next_sibling.get_text(strip=True))\n        next_sibling = next_sibling.find_next_sibling()\n\n    doc_dict[tag.get_text(strip=True)] = " ".join(content)\n```\n\nBy adding `tag.name == \'h1\'` to the while loop condition, it will include the p tags after the h1 tag as well.')

In [ ]:
working_code = """from bs4 import BeautifulSoup, Comment
import requests

# Load the webpage content
url = 'http://your-website-url.com'
r = requests.get(url)
html_content = r.text

# Create a BeautifulSoup object
soup = BeautifulSoup(html_content, 'html.parser')

# Remove unwanted tags: script, style, [document], head, title
for element in soup(["script", "style", "head", "title", "[document]"]):
    element.decompose()

# Also remove HTML comments
for element in soup.find_all(text=lambda text: isinstance(text, Comment)):
    element.extract()

# Define the tags to find
tags = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']
found_tags = soup.find_all(tags)

# Extract tags and their associated content into a dictionary
doc_dict = {}
for tag in found_tags:
    content = []
    next_sibling = tag.find_next_sibling()

    while next_sibling and next_sibling.name not in tags:
        if next_sibling.name not in ["script", "style", "head", "title", "[document]"]:
            content.append(next_sibling.get_text(strip=True))
        next_sibling = next_sibling.find_next_sibling()

    doc_dict[tag.get_text(strip=True)] = " ".join(content)

# Print the resulting dictionary
for k, v in doc_dict.items():
    print(f'{k}: {v}')
"""

prompt = f"""The following code was created by GPT-4 and successfully achives this goal: I only care about what a person reading the page will read. So just get me the headers and actual content text content that a human would see. The current code I tried gets things like 'document.documentElement.className="client-js ' which is obviously js code that the user would not read. add everything to a dict in the order you find them.

However, it does not find the p tag for the h1 tag. Please correct it so that
it finds the p tag after the h1 tag

Working code: ####
{working_code}
####

html soup: ####
{broken_html}
####
"""
output = llm_16k.predict(prompt)
pprint(output)

In [ ]:
working_code = """from bs4 import BeautifulSoup, Comment
import requests

# Load the webpage content
url = 'http://your-website-url.com'
r = requests.get(url)
html_content = r.text

# Create a BeautifulSoup object
soup = BeautifulSoup(html_content, 'html.parser')

# Remove unwanted tags: script, style, [document], head, title
for element in soup(["script", "style", "head", "title", "[document]"]):
    element.decompose()

# Also remove HTML comments
for element in soup.find_all(text=lambda text: isinstance(text, Comment)):
    element.extract()

# Define the tags to find
tags = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']
found_tags = soup.find_all(tags)

# Extract tags and their associated content into a dictionary
doc_dict = {}
for tag in found_tags:
    content = []
    next_sibling = tag.find_next_sibling()

    while next_sibling and next_sibling.name not in tags:
        if next_sibling.name not in ["script", "style", "head", "title", "[document]"]:
            content.append(next_sibling.get_text(strip=True))
        next_sibling = next_sibling.find_next_sibling()

    doc_dict[tag.get_text(strip=True)] = " ".join(content)

# Print the resulting dictionary
for k, v in doc_dict.items():
    print(f'{k}: {v}')
"""

prompt = f"""The following code finds all the p tags after header tags. But it does not find the p tags for the h1 tag. Why does it not find it?

Code: ####
{working_code}
####

html soup: ####
{broken_html}
####
"""
output = llm_16k.predict(prompt)
pprint(output)

In [ ]:
soup

In [ ]:
Introduction
The DPE mechanism
Underlying network
Phase shifts
Selection and variation
System memory
Examples
Social networks
Socio-economics

In [ ]:
shortest_article = all_articles[2]
shortest_article.page_content

In [ ]:
llm_16k.predict(f"Summarize the following text: {shortest_article.page_content}")

In [ ]:
# llm_default.predict(f'Translate the following text into German: {all_articles[3].page_content}')
output = llm_16k.predict(f'Translate the following text into German: {all_articles[3].page_content}')

In [ ]:
num_tokens_from_string(output)

In [ ]:
wikipedia_articles[3]

In [ ]:
len(all_articles[3].page_content)

enc = tiktoken.encoding_for_model('gpt-3.5-turbo')
len(all_articles[3].page_content.split(' '))

In [ ]:
num_tokens_from_string(all_articles[3].page_content)